# Import Libraries and Files

In [1]:
# Import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix
import catboost


from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn import tree
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from catboost import Pool, sum_models
from sklearn.model_selection import cross_val_score
from sklearn.metrics import balanced_accuracy_score, accuracy_score
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.model_selection import StratifiedKFold


In [3]:
# Import preprocessed test file

processed_test = pd.read_csv('Saved Files/Top50_catboost')
processed_test.head()

,amount_tsh,gps_height,longitude,latitude,basin,region,region_code,district_code,population,scheme_management,...,management,payment,water_quality,quantity,source,waterpoint_type,well_age,top_funded,top_installers,top_lga
0,0.0,1996,35.290799,-4.059696,Internal,Manyara,21,3,321,Parastatal,...,parastatal,never pay,soft,seasonal,rainwater harvesting,other,1,Dmdd,other,other
1,0.0,1569,36.656709,-3.309214,Pangani,Arusha,2,2,300,VWC,...,vwc,never pay,soft,insufficient,spring,communal standpipe,13,Government Of Tanzania,DWE,Arusha Rural
2,0.0,1567,34.767863,-5.004344,Internal,Singida,13,2,500,VWC,...,vwc,never pay,soft,insufficient,rainwater harvesting,other,3,other,other,Singida Rural
3,0.0,267,38.058046,-9.418672,Ruvuma / Southern Coast,Lindi,80,43,250,VWC,...,vwc,unknown,soft,dry,shallow well,other,26,other,other,other
4,500.0,1260,35.006123,-10.950412,Ruvuma / Southern Coast,Ruvuma,10,3,60,Water Board,...,water board,pay monthly,soft,enough,spring,communal standpipe,13,other,other,Mbinga


In [44]:
processed_test.dtypes

amount_tsh               float64
gps_height                 int64
longitude                float64
latitude                 float64
basin                     object
region                    object
region_code                int64
district_code              int64
population                 int64
scheme_management         object
extraction_type_class     object
management                object
payment                   object
water_quality             object
quantity                  object
source                    object
waterpoint_type           object
well_age                   int64
top_funded                object
top_installers            object
top_lga                   object
dtype: object

In [4]:
processed_test.isnull().sum()

amount_tsh               0
gps_height               0
longitude                0
latitude                 0
basin                    0
region                   0
region_code              0
district_code            0
population               0
scheme_management        0
extraction_type_class    0
management               0
payment                  0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
well_age                 0
top_funded               0
top_installers           0
top_lga                  0
dtype: int64

In [5]:
# Import processed training data 

df_train = pd.read_csv('Saved Files/experimental50_training_set')
df_train.head()

,amount_tsh,gps_height,longitude,latitude,basin,region,region_code,district_code,population,scheme_management,...,payment,water_quality,quantity,source,waterpoint_type,well_age,status_group,top_funded,top_installers,top_lga
0,6000.0,1390,34.938093,-9.856322,Lake Nyasa,Iringa,11,5,109,VWC,...,pay annually,soft,enough,spring,communal standpipe,12,functional,Roman,other,Ludewa
1,0.0,1399,34.698766,-2.147466,Lake Victoria,Mara,20,2,280,other,...,never pay,soft,insufficient,rainwater harvesting,communal standpipe,3,functional,other,other,Serengeti
2,25.0,686,37.460664,-3.821329,Pangani,Manyara,21,4,250,VWC,...,pay per bucket,soft,enough,dam,communal standpipe multiple,4,functional,other,World Vision,other
3,0.0,263,38.486161,-11.155298,Ruvuma / Southern Coast,Mtwara,90,63,58,VWC,...,never pay,soft,dry,machine dbh,communal standpipe multiple,27,non functional,Unicef,UNICEF,other
4,0.0,0,31.130847,-1.825359,Lake Victoria,Kagera,18,1,0,other,...,never pay,soft,seasonal,rainwater harvesting,communal standpipe,12,functional,other,other,Karagwe


In [6]:
df_train.isnull().sum()

amount_tsh               0
gps_height               0
longitude                0
latitude                 0
basin                    0
region                   0
region_code              0
district_code            0
population               0
scheme_management        0
extraction_type_class    0
management               0
payment                  0
water_quality            0
quantity                 0
source                   0
waterpoint_type          0
well_age                 0
status_group             0
top_funded               0
top_installers           0
top_lga                  0
dtype: int64

In [7]:
# Import submission format file

model_submit = pd.read_csv('DrivenData Files/SubmissionFormat.csv')
model_submit.head()

,id,status_group
0,50785,predicted label
1,51630,predicted label
2,17168,predicted label
3,45559,predicted label
4,49871,predicted label


In [8]:
model_submit.drop(['status_group'], axis = 1, inplace = True)
model_submit.head()

,id
0,50785
1,51630
2,17168
3,45559
4,49871


# Split Data and Train a Naive Model

In [9]:
X = df_train.drop(['status_group'], axis = 1)
y = df_train['status_group']

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [11]:
categorical_features_indices = np.where(X.dtypes != np.float)[0]

In [48]:
catmodel = CatBoostClassifier(loss_function = 'MultiClass', cat_features = categorical_features_indices)

In [26]:
cbmodel = CatBoostClassifier(iterations = 100, random_seed = 42, loss_function = 'MultiClass')

In [27]:
cbmodel.fit(X_train, y_train, cat_features= categorical_features_indices, 
            eval_set=(X_test, y_test), verbose = False, plot=False)

In [28]:
test_pool = Pool(data = processed_test, cat_features = categorical_features_indices)
preds = cbmodel.predict_proba(test_pool)
print(preds)

[[0.71611366 0.09748756 0.18639878]
 [0.59899232 0.07557467 0.32543301]
 [0.80331969 0.08264638 0.11403393]
 ...
 [0.80122293 0.10333876 0.09543831]
 [0.65937664 0.16117267 0.17945069]
 [0.07270038 0.00402509 0.92327453]]


In [29]:
preds_class = cbmodel.predict(test_pool)
print(preds_class)

[['functional']
 ['functional']
 ['functional']
 ...
 ['functional']
 ['functional']
 ['non functional']]


In [33]:
# Format the test predictions into a pandas dataframe

test_pred_df = pd.DataFrame(data = preds_class)

# Name the outcome label column

test_pred_df = test_pred_df.rename(columns = {0: 'status_group'})

# Concatenate the test labels and submission format file

test_pred_df = pd.concat([model_submit, test_pred_df], axis = 1)

In [34]:
test_pred_df['status_group'].value_counts()

functional                 10091
non functional              4569
functional needs repair      190
Name: status_group, dtype: int64

In [39]:
# Top 50 Model, CatBoost Classifier: Score = 0.7643

test_pred_df.to_csv('test_set_predictions_14', index = False)

In [42]:
grid = {'iterations': [100, 250, 500, 1000],
        'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

In [50]:
grid_search_result = catmodel.randomized_search(grid, X = X, y = y, search_by_train_test_split = True,
                                         refit = True, verbose = 1000)

Custom logger is already specified. Specify more than one logger at same time is not thread safe.

0:	learn: 1.0282439	test: 1.0283927	best: 1.0283927 (0)	total: 314ms	remaining: 2m 36s
1:	learn: 0.9707377	test: 0.9701067	best: 0.9701067 (1)	total: 526ms	remaining: 2m 10s
2:	learn: 0.9256767	test: 0.9253282	best: 0.9253282 (2)	total: 877ms	remaining: 2m 25s
3:	learn: 0.8883328	test: 0.8880277	best: 0.8880277 (3)	total: 1.18s	remaining: 2m 26s
4:	learn: 0.8560445	test: 0.8552521	best: 0.8552521 (4)	total: 1.45s	remaining: 2m 23s
5:	learn: 0.8268599	test: 0.8252412	best: 0.8252412 (5)	total: 1.73s	remaining: 2m 22s
6:	learn: 0.8029167	test: 0.8008289	best: 0.8008289 (6)	total: 2.01s	remaining: 2m 21s
7:	learn: 0.7843135	test: 0.7824574	best: 0.7824574 (7)	total: 2.25s	remaining: 2m 18s
8:	learn: 0.7648715	test: 0.7623665	best: 0.7623665 (8)	total: 2.54s	remaining: 2m 18s
9:	learn: 0.7489026	test: 0.7458572	best: 0.7458572 (9)	total: 2.86s	remaining: 2m 19s
10:	learn: 0.7349956	test: 0.7314363	best: 0.7314363 (10)	total: 3.19s	remaining: 2m 21s
11:	learn: 0.7235415	test: 0.7198310	best

93:	learn: 0.5718947	test: 0.5637168	best: 0.5637168 (93)	total: 29.9s	remaining: 2m 9s
94:	learn: 0.5717655	test: 0.5636189	best: 0.5636189 (94)	total: 30.2s	remaining: 2m 8s
95:	learn: 0.5713970	test: 0.5633438	best: 0.5633438 (95)	total: 30.6s	remaining: 2m 8s
96:	learn: 0.5711899	test: 0.5631236	best: 0.5631236 (96)	total: 30.9s	remaining: 2m 8s
97:	learn: 0.5707939	test: 0.5627467	best: 0.5627467 (97)	total: 31.2s	remaining: 2m 7s
98:	learn: 0.5704853	test: 0.5624574	best: 0.5624574 (98)	total: 32s	remaining: 2m 9s
99:	learn: 0.5702784	test: 0.5623648	best: 0.5623648 (99)	total: 33.2s	remaining: 2m 12s
100:	learn: 0.5700003	test: 0.5620245	best: 0.5620245 (100)	total: 34.3s	remaining: 2m 15s
101:	learn: 0.5693460	test: 0.5614566	best: 0.5614566 (101)	total: 34.7s	remaining: 2m 15s
102:	learn: 0.5686278	test: 0.5607660	best: 0.5607660 (102)	total: 35s	remaining: 2m 14s
103:	learn: 0.5683594	test: 0.5603973	best: 0.5603973 (103)	total: 36s	remaining: 2m 17s
104:	learn: 0.5678973	tes

184:	learn: 0.5503038	test: 0.5444525	best: 0.5444525 (184)	total: 1m 7s	remaining: 1m 54s
185:	learn: 0.5499531	test: 0.5440271	best: 0.5440271 (185)	total: 1m 7s	remaining: 1m 54s
186:	learn: 0.5498672	test: 0.5439299	best: 0.5439299 (186)	total: 1m 8s	remaining: 1m 55s
187:	learn: 0.5495894	test: 0.5436596	best: 0.5436596 (187)	total: 1m 9s	remaining: 1m 54s
188:	learn: 0.5494698	test: 0.5435878	best: 0.5435878 (188)	total: 1m 9s	remaining: 1m 54s
189:	learn: 0.5493347	test: 0.5435150	best: 0.5435150 (189)	total: 1m 9s	remaining: 1m 53s
190:	learn: 0.5489436	test: 0.5431076	best: 0.5431076 (190)	total: 1m 10s	remaining: 1m 53s
191:	learn: 0.5487638	test: 0.5430722	best: 0.5430722 (191)	total: 1m 10s	remaining: 1m 52s
192:	learn: 0.5486642	test: 0.5430126	best: 0.5430126 (192)	total: 1m 10s	remaining: 1m 52s
193:	learn: 0.5485676	test: 0.5429586	best: 0.5429586 (193)	total: 1m 11s	remaining: 1m 51s
194:	learn: 0.5483789	test: 0.5427790	best: 0.5427790 (194)	total: 1m 11s	remaining: 1

274:	learn: 0.5393712	test: 0.5362618	best: 0.5362618 (274)	total: 1m 36s	remaining: 1m 18s
275:	learn: 0.5393041	test: 0.5362340	best: 0.5362340 (275)	total: 1m 36s	remaining: 1m 18s
276:	learn: 0.5392817	test: 0.5362184	best: 0.5362184 (276)	total: 1m 36s	remaining: 1m 17s
277:	learn: 0.5392423	test: 0.5362145	best: 0.5362145 (277)	total: 1m 37s	remaining: 1m 17s
278:	learn: 0.5391436	test: 0.5361601	best: 0.5361601 (278)	total: 1m 37s	remaining: 1m 17s
279:	learn: 0.5390875	test: 0.5361059	best: 0.5361059 (279)	total: 1m 37s	remaining: 1m 16s
280:	learn: 0.5389400	test: 0.5359212	best: 0.5359212 (280)	total: 1m 37s	remaining: 1m 16s
281:	learn: 0.5387480	test: 0.5356232	best: 0.5356232 (281)	total: 1m 38s	remaining: 1m 15s
282:	learn: 0.5387144	test: 0.5356171	best: 0.5356171 (282)	total: 1m 38s	remaining: 1m 15s
283:	learn: 0.5386194	test: 0.5355624	best: 0.5355624 (283)	total: 1m 38s	remaining: 1m 15s
284:	learn: 0.5385364	test: 0.5354891	best: 0.5354891 (284)	total: 1m 39s	remain

...

596:	learn: 0.4355264	test: 0.5116115	best: 0.5116115 (596)	total: 44m 31s	remaining: 30m 3s
597:	learn: 0.4354152	test: 0.5115828	best: 0.5115828 (597)	total: 44m 35s	remaining: 29m 58s
598:	learn: 0.4352711	test: 0.5115438	best: 0.5115438 (598)	total: 44m 39s	remaining: 29m 53s
599:	learn: 0.4351354	test: 0.5115031	best: 0.5115031 (599)	total: 44m 43s	remaining: 29m 49s
600:	learn: 0.4349743	test: 0.5114904	best: 0.5114904 (600)	total: 44m 48s	remaining: 29m 44s
601:	learn: 0.4348322	test: 0.5114602	best: 0.5114602 (601)	total: 44m 53s	remaining: 29m 40s
602:	learn: 0.4347312	test: 0.5114342	best: 0.5114342 (602)	total: 44m 57s	remaining: 29m 36s
603:	learn: 0.4346230	test: 0.5113918	best: 0.5113918 (603)	total: 45m 2s	remaining: 29m 31s
604:	learn: 0.4344245	test: 0.5113664	best: 0.5113664 (604)	total: 45m 6s	remaining: 29m 27s
605:	learn: 0.4343040	test: 0.5113352	best: 0.5113352 (605)	total: 45m 11s	remaining: 29m 22s
606:	learn: 0.4341859	test: 0.5113085	best: 0.5113085 (606)	tot

684:	learn: 0.4233653	test: 0.5092731	best: 0.5092731 (684)	total: 51m 11s	remaining: 23m 32s
685:	learn: 0.4232316	test: 0.5092448	best: 0.5092448 (685)	total: 51m 16s	remaining: 23m 28s
686:	learn: 0.4230655	test: 0.5092335	best: 0.5092335 (686)	total: 51m 20s	remaining: 23m 23s
687:	learn: 0.4229263	test: 0.5092077	best: 0.5092077 (687)	total: 51m 25s	remaining: 23m 19s
688:	learn: 0.4228239	test: 0.5091806	best: 0.5091806 (688)	total: 51m 29s	remaining: 23m 14s
689:	learn: 0.4226819	test: 0.5091585	best: 0.5091585 (689)	total: 51m 33s	remaining: 23m 9s
690:	learn: 0.4225776	test: 0.5091478	best: 0.5091478 (690)	total: 51m 38s	remaining: 23m 5s
691:	learn: 0.4224563	test: 0.5091239	best: 0.5091239 (691)	total: 51m 42s	remaining: 23m
692:	learn: 0.4222825	test: 0.5090963	best: 0.5090963 (692)	total: 51m 46s	remaining: 22m 56s
693:	learn: 0.4221851	test: 0.5090762	best: 0.5090762 (693)	total: 51m 51s	remaining: 22m 52s
694:	learn: 0.4220511	test: 0.5090783	best: 0.5090762 (693)	total:

772:	learn: 0.4119622	test: 0.5076029	best: 0.5076029 (772)	total: 57m 53s	remaining: 16m 59s
773:	learn: 0.4117867	test: 0.5075684	best: 0.5075684 (773)	total: 58m 1s	remaining: 16m 56s
774:	learn: 0.4116505	test: 0.5075476	best: 0.5075476 (774)	total: 58m 8s	remaining: 16m 52s
775:	learn: 0.4115127	test: 0.5075270	best: 0.5075270 (775)	total: 58m 16s	remaining: 16m 49s
776:	learn: 0.4114255	test: 0.5075072	best: 0.5075072 (776)	total: 58m 22s	remaining: 16m 45s
777:	learn: 0.4113488	test: 0.5074919	best: 0.5074919 (777)	total: 58m 28s	remaining: 16m 41s
778:	learn: 0.4112395	test: 0.5074740	best: 0.5074740 (778)	total: 58m 34s	remaining: 16m 37s
779:	learn: 0.4111412	test: 0.5074493	best: 0.5074493 (779)	total: 58m 40s	remaining: 16m 32s
780:	learn: 0.4110444	test: 0.5074390	best: 0.5074390 (780)	total: 58m 45s	remaining: 16m 28s
781:	learn: 0.4109263	test: 0.5074070	best: 0.5074070 (781)	total: 58m 50s	remaining: 16m 24s
782:	learn: 0.4108016	test: 0.5073778	best: 0.5073778 (782)	to

860:	learn: 0.4013492	test: 0.5059704	best: 0.5059704 (860)	total: 1h 5m 6s	remaining: 10m 30s
861:	learn: 0.4012585	test: 0.5059570	best: 0.5059570 (861)	total: 1h 5m 10s	remaining: 10m 26s
862:	learn: 0.4011969	test: 0.5059450	best: 0.5059450 (862)	total: 1h 5m 15s	remaining: 10m 21s
863:	learn: 0.4010454	test: 0.5059158	best: 0.5059158 (863)	total: 1h 5m 20s	remaining: 10m 17s
864:	learn: 0.4008717	test: 0.5059045	best: 0.5059045 (864)	total: 1h 5m 27s	remaining: 10m 13s
865:	learn: 0.4007882	test: 0.5058909	best: 0.5058909 (865)	total: 1h 5m 32s	remaining: 10m 8s
866:	learn: 0.4006709	test: 0.5058884	best: 0.5058884 (866)	total: 1h 5m 40s	remaining: 10m 4s
867:	learn: 0.4005650	test: 0.5058758	best: 0.5058758 (867)	total: 1h 5m 44s	remaining: 9m 59s
868:	learn: 0.4004505	test: 0.5058601	best: 0.5058601 (868)	total: 1h 5m 49s	remaining: 9m 55s
869:	learn: 0.4003211	test: 0.5058532	best: 0.5058532 (869)	total: 1h 5m 53s	remaining: 9m 50s
870:	learn: 0.4002056	test: 0.5058430	best: 0.

947:	learn: 0.3914570	test: 0.5047322	best: 0.5047322 (947)	total: 1h 12m 10s	remaining: 3m 57s
948:	learn: 0.3913240	test: 0.5047327	best: 0.5047322 (947)	total: 1h 12m 14s	remaining: 3m 52s
949:	learn: 0.3912521	test: 0.5047205	best: 0.5047205 (949)	total: 1h 12m 19s	remaining: 3m 48s
950:	learn: 0.3910260	test: 0.5047133	best: 0.5047133 (950)	total: 1h 12m 23s	remaining: 3m 43s
951:	learn: 0.3908690	test: 0.5046876	best: 0.5046876 (951)	total: 1h 12m 28s	remaining: 3m 39s
952:	learn: 0.3907023	test: 0.5046726	best: 0.5046726 (952)	total: 1h 12m 32s	remaining: 3m 34s
953:	learn: 0.3905829	test: 0.5046611	best: 0.5046611 (953)	total: 1h 12m 36s	remaining: 3m 30s
954:	learn: 0.3904920	test: 0.5046371	best: 0.5046371 (954)	total: 1h 12m 41s	remaining: 3m 25s
955:	learn: 0.3903762	test: 0.5046369	best: 0.5046369 (955)	total: 1h 12m 46s	remaining: 3m 20s
956:	learn: 0.3902191	test: 0.5046284	best: 0.5046284 (956)	total: 1h 12m 51s	remaining: 3m 16s
957:	learn: 0.3900559	test: 0.5046182	be

In [51]:
grid_search_result

{'params': {'depth': 10,
  'l2_leaf_reg': 3,
  'iterations': 1000,
  'learning_rate': 0.03},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               

In [52]:
cbmodel2 = CatBoostClassifier(depth = 10, l2_leaf_reg = 3, iterations = 1000,
                             learning_rate = 0.03, random_seed = 42, 
                             loss_function = 'MultiClass')

In [53]:
cbmodel2.fit(X_train, y_train, cat_features= categorical_features_indices, 
            eval_set=(X_test, y_test), verbose = False, plot=False)

In [54]:
test_pool = Pool(data = processed_test, cat_features = categorical_features_indices)
preds2 = cbmodel2.predict_proba(test_pool)
print(preds2)

[[0.43091042 0.09221949 0.47687009]
 [0.71370563 0.0439198  0.24237457]
 [0.92459533 0.03989819 0.03550649]
 ...
 [0.89569373 0.06102181 0.04328447]
 [0.865638   0.06674429 0.0676177 ]
 [0.0323177  0.00402108 0.96366122]]


In [55]:
preds_class2 = cbmodel2.predict(test_pool)
print(preds_class2)

[['non functional']
 ['functional']
 ['functional']
 ...
 ['functional']
 ['functional']
 ['non functional']]


In [58]:
# Format the test predictions into a pandas dataframe

test_pred_df2 = pd.DataFrame(data = preds_class2)

# Name the outcome label column

test_pred_df2 = test_pred_df2.rename(columns = {0: 'status_group'})

# Concatenate the test labels and submission format file

test_pred_df2 = pd.concat([model_submit, test_pred_df2], axis = 1)

In [59]:
test_pred_df2['status_group'].value_counts()

functional                 9810
non functional             4765
functional needs repair     275
Name: status_group, dtype: int64

In [60]:
# Top 50 Model, CatBoost Classifier: Score = 0.7945

test_pred_df2.to_csv('test_set_predictions_15', index = False)

## Experimenting with hyper-parameters

In [12]:
cbmodel = CatBoostClassifier(depth = 6, iterations = 1000, random_seed = 42, loss_function = 'MultiClass')

In [13]:
cbmodel.fit(X_train, y_train, cat_features= categorical_features_indices, 
            eval_set=(X_test, y_test), verbose = False, plot=False)

In [15]:
print('Training accuracy:{}'.format(cbmodel.score(X_train, y_train)))
print('Testing accuracy:{}'.format(cbmodel.score(X_test, y_test)))

Training accuracy:0.8329335016835017
Testing accuracy:0.7981481481481482


In [16]:
cbmodel2 = CatBoostClassifier(depth = 10, iterations = 1000, random_seed = 42, loss_function = 'MultiClass')

In [17]:
cbmodel2.fit(X_train, y_train, cat_features= categorical_features_indices, 
            eval_set=(X_test, y_test), verbose = False, plot=False)

In [18]:
s

Training accuracy:0.8431186868686869
Testing accuracy:0.7994107744107745


In [19]:
cbmodel3 = CatBoostClassifier(depth = 10, iterations = 500, random_seed = 42, loss_function = 'MultiClass')

In [20]:
cbmodel3.fit(X_train, y_train, cat_features= categorical_features_indices, 
            eval_set=(X_test, y_test), verbose = False, plot=False)

In [21]:
print('Training accuracy:{}'.format(cbmodel3.score(X_train, y_train)))
print('Testing accuracy:{}'.format(cbmodel3.score(X_test, y_test)))

Training accuracy:0.844023569023569
Testing accuracy:0.7998316498316498


In [22]:
test_pool = Pool(data = processed_test, cat_features = categorical_features_indices)
preds = cbmodel3.predict(test_pool)
print(preds)

[['non functional']
 ['functional']
 ['functional']
 ...
 ['functional']
 ['functional']
 ['non functional']]


In [23]:
# Format the test predictions into a pandas dataframe

test_pred_df = pd.DataFrame(data = preds)

# Name the outcome label column

test_pred_df = test_pred_df.rename(columns = {0: 'status_group'})

# Concatenate the test labels and submission format file

test_pred_df = pd.concat([model_submit, test_pred_df], axis = 1)

In [24]:
test_pred_df['status_group'].value_counts()

functional                 9670
non functional             4851
functional needs repair     329
Name: status_group, dtype: int64

In [25]:
# Top 50 Model, CatBoost Classifier: Score = 0.7945

test_pred_df.to_csv('test_set_predictions_20', index = False)

In [ ]:
{'params': {'depth': 10,
  'l2_leaf_reg': 3,
  'iterations': 1000,
  'learning_rate': 0.03},

In [ ]:
cv_params = cbmodel.get_params()
cv_params.update({
    'loss_function': metrics.Logloss()
})
cv_data = cv(
    Pool(X, y, cat_features=categorical_features_indices),
    cv_params,
    plot=True
)